## Imports, seed and load data

In [ ]:
from pytorch_lightning import callbacks
import torch
from torch import nn, optim
from torch.nn import functional as F
import pytorch_lightning as pl
from torchmetrics.functional import accuracy

import pandas as pd
import numpy as np
import os
from typing import Union, Tuple
import wandb

try:
    from country_region import *
    from data_processing import *
except Exception as e:
    print(e)

In [8]:
def fix_all_seeds(seed):
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

fix_all_seeds(42)

In [9]:
meta = pd.read_csv('4000_PCS_human_origins/v44.3_HO_public.anno', sep='\t')
pcs = pd.read_csv('4000_PCS_human_origins/pcs.txt', sep='\t')

## FFNN Model

In [24]:
# Simple ffnn
class ffnn(pl.LightningModule):
    def __init__(self, input_size: int=100, layer_1: int=2000, layer_2: int=1500, layer_3: int=1000, layer_4: int=500,
                 dropout: float=0.25,
                 lr: float=1.5e-4, weight_decay: float=1e-4):
        super().__init__()
        
        self.lr = lr
        self.weight_decay = weight_decay
        
        self.fc1 = nn.Linear(
            in_features=input_size,
            out_features=layer_1
        )

        self.fc2 = nn.Linear(
            in_features=layer_1,
            out_features=layer_2
        )
        
        self.fc3 = nn.Linear(
            in_features=layer_2,
            out_features=layer_3
        )
        
        self.fc4 = nn.Linear(
            in_features=layer_3,
            out_features=layer_4
        )

        self.fc_out = nn.Linear(
            in_features=layer_4,
            out_features=11  # Number of regions (in the dataset = 12, originally 14)  # 143 countries
        )

        self.dropout = nn.Dropout(dropout)
        self.batchnorm1 = nn.BatchNorm1d(layer_1)
        self.batchnorm2 = nn.BatchNorm1d(layer_2)
        self.batchnorm3 = nn.BatchNorm1d(layer_3)
        self.batchnorm4 = nn.BatchNorm1d(layer_4)

    def forward(self, x):
        z = self.fc1(x)
        z = F.relu(z)
        z = self.batchnorm1(z)
        z = self.dropout(z)

        z = self.fc2(z)
        z = F.relu(z)
        z = self.batchnorm2(z)
        z = self.dropout(z)
        
        z = self.fc3(z)
        z = F.relu(z)
        z = self.batchnorm3(z)
        z = self.dropout(z)
        
        z = self.fc4(z)
        z = F.relu(z)
        z = self.batchnorm4(z)
        z = self.dropout(z)

        out = self.fc_out(z)

        return out
    
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        return optimizer
    

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = accuracy(logits, y)
        self.log('train/loss', loss, on_epoch=True)
        self.log('train/acc', acc, on_epoch=True)
        return loss
    
    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = accuracy(logits, y)
        self.log('val/loss', loss, on_epoch=True)
        self.log('val/acc', acc, on_epoch=True)
        return loss

## Load hyperparameters found through optuna

In [ ]:
INPUT_SIZE = 100

In [ ]:
hyperparameters = yaml.load(f'Project/best_hyperparameters/input_{INPUT_SIZE}.yaml')

batch_size = hyperparameters['batch_size']
layer_1    = hyperparameters['layer_1']
layer_2    = hyperparameters['layer_2']
layer_3    = hyperparameters['layer_3']
layer_4    = hyperparameters['layer_4']
dropout    = round(hyperparameters['dropout'], 5)

## Prepare data

In [ ]:
x, y, idx_to_country = prepare_data(pcs, meta, num_of_pcs=INPUT_SIZE)
train_loader, val_loader = create_dataloaders(x, y, batch_size=batch_size, train_shuffle=True)

### Training using PyTorch Lightning

In [11]:
gpu = 1 if torch.cuda.is_available() else 0

wandb_logger = WandbLogger(project='02456DeepLearning_project', name=f'input-{INPUT_SIZE}-with_ancient')

trainer = pl.Trainer(
    logger=wandb_logger,
    # log_every_n_steps=5,
    gpus=gpu, 
    max_epochs=100
)
    
trainer.logger.log_hyperparams(hyperparameters)

model = ffnn(input_size=100)
trainer.fit(model, train_loader, val_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type        | Params
-------------------------------------------
0 | fc1        | Linear      | 202 K 
1 | fc2        | Linear      | 3.0 M 
2 | fc3        | Linear      | 1.5 M 
3 | fc4        | Linear      | 500 K 
4 | fc_out     | Linear      | 6.0 K 
5 | dropout    | Dropout     | 0     
6 | batchnorm1 | BatchNorm1d | 4.0 K 
7 | batchnorm2 | BatchNorm1d | 3.0 K 
8 | batchnorm3 | BatchNorm1d | 2.0 K 
9 | batchnorm4 | BatchNorm1d | 1.0 K 
-------------------------------------------
5.2 M     Trainable params
0         Non-trainable params
5.2 M     Total params
20.884    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

d:\github\02456-deep-learning\venv\lib\site-packages\pytorch_lightning\trainer\data_loading.py:110: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
d:\github\02456-deep-learning\venv\lib\site-packages\pytorch_lightning\trainer\data_loading.py:110: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

d:\github\02456-deep-learning\venv\lib\site-packages\pytorch_lightning\trainer\trainer.py:685: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [11]:
trainer.callback_metrics

NameError: name 'trainer' is not defined